<a href="https://colab.research.google.com/github/mansikhobragade/Helpdesk/blob/main/Copy_of_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import zipfile

import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
zip_file_path = ('/content/drive/MyDrive/B. Disease Grading.zip')
target_folder = ('/content/drive/MyDrive/')

In [ ]:
with zipfile.ZipFile(zip_file_path,'r') as zip_ref:
  zip_ref.extractall(target_folder)

In [ ]:
extracted_folder = os.path.splitext(zip_file_path)[0] #Remove the .zip extension
contents = os.listdir(extracted_folder)
print("contents of the extracted folder:")
for item in contents:
  print (item)


In [ ]:
groundtruth_folder = os.path.join(extracted_folder, '2. Groundtruths')
groundtruth_contents = os.listdir(groundtruth_folder)

print("\nFiles under 'Groundtruths':")
print(groundtruth_contents)
for csv_file in groundtruth_contents:
  if csv_file.endswith('.csv'):
    csv_path = os.path.join(groundtruth_folder,csv_file)
    df = pd.read_csv(csv_path)
    print(f"\nHead of '{csv_file}':")
    print(df.head())

In [ ]:
import os
print(os.listdir())

In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2
from sklearn.utils import class_weight, shuffle
from keras.losses import binary_crossentropy
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
#from keras.applications.resnet50 import ResNet50
#from keras.applications.resnet50 import preprocess_input
import keras.backend as K
import tensorflow as tf
from sklearn.metrics import f1_score, fbeta_score
from keras.utils import Sequence
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

WORKERS = 2
CHANNEL = 3

import warnings
warnings.filterwarnings("ignore")
IMG_SIZE = 512
NUM_CLASSES = 5
SEED = 77
TRAIN_NUM = 1000 # use 1000 when you just want to explore new idea, use -1 for full train

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/B. Disease Grading/2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv')
df_test = pd.read_csv('/content/drive/MyDrive/B. Disease Grading/2. Groundtruths/b. IDRiD_Disease Grading_Testing Labels.csv')

x = df_train['Image name']
y = df_train['Retinopathy grade']

x, y = shuffle(x, y, random_state=SEED)

In [ ]:
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.20, stratify=y, random_state=SEED)
print(train_x.shape, train_y.shape, valid_x.shape, valid_y.shape)


In [ ]:
train_y.hist()
valid_y.hist()

In [ ]:
pwd

In [ ]:
%%time
!pwd
fig = plt.figure(figsize=(15, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['Retinopathy grade'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"/content/drive/MyDrive/B. Disease Grading/1. Original Images/a. Training Set/{row['Image name']}.jpg"
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))

        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['Image name']) )

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# Define the CLAHE parameters
clahe = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8, 8))

# Specify the paths to the dataset
dataset_path = "/content/drive/MyDrive/B. Disease Grading"
train_images_path = os.path.join(dataset_path, "1. Original Images/a. Training Set/")
train_csv_path = os.path.join(dataset_path, "2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv")

# Read the train CSV file
train_df = pd.read_csv(train_csv_path)

# Define the list of classes
class_names = ["class_0", "class_1", "class_2", "class_3", "class_4"]

# Preprocess 1 random image from each class
num_samples = 2

for class_name in class_names:
    class_df = train_df[train_df["Retinopathy grade"] == class_names.index(class_name)]
    class_samples = class_df.sample(n=num_samples, random_state=42)

    for _, row in class_samples.iterrows():
        image_id = row["Image name"]
        image_path = os.path.join(train_images_path, image_id + ".jpg")

        # Load the image
        image = cv2.imread(image_path, 0)  # Load as grayscale image

        # Apply CLAHE
        image_clahe = clahe.apply(image)

        # Convert back to BGR color space for displaying with matplotlib
        image_clahe_bgr = cv2.cvtColor(image_clahe, cv2.COLOR_GRAY2BGR)


        # Plot the original and CLAHE processed images
        fig, axes = plt.subplots(1, 2, figsize=(5, 5))
        axes[0].imshow(image, cmap="gray")
        axes[0].set_title(f"Original img-{class_name}")
        axes[0].axis("off")
        axes[1].imshow(image_clahe_bgr)
        axes[1].set_title(f"CLAHE img- {class_name}")
        axes[1].axis("off")
        plt.show()


In [ ]:
import cv2
import os

# Define the CLAHE parameters
clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(8, 8))

# Specify the paths to the directories
dataset_directory = "/content/drive/MyDrive/B. Disease Grading"
train_images_directory = os.path.join(dataset_directory, "1. Original Images/a. Training Set/")
test_images_directory = os.path.join(dataset_directory, "1. Original Images/b. Testing Set/")
output_directory = os.path.join(dataset_directory, "1. Original Images/clahe_images")
train_output_directory = os.path.join(output_directory, "clahe_train_images")
test_output_directory = os.path.join(output_directory, "clahe_test_images")

# Create the output directories if they don't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
if not os.path.exists(train_output_directory):
    os.makedirs(train_output_directory)
if not os.path.exists(test_output_directory):
    os.makedirs(test_output_directory)

# Process images in train_images directory
for filename in os.listdir(train_images_directory):
    if filename.endswith(".jpg"):
        # Read the image
        image_path = os.path.join(train_images_directory, filename)
        image = cv2.imread(image_path, 0)  # Load as grayscale image

        # Apply CLAHE
        image_clahe = clahe.apply(image)

        # Save the CLAHE processed image
        output_path = os.path.join(train_output_directory, filename)
        cv2.imwrite(output_path, image_clahe)

# Process images in test_images directory
for filename in os.listdir(test_images_directory):
    if filename.endswith(".png"):
        # Read the image
        image_path = os.path.join(test_images_directory, filename)
        image = cv2.imread(image_path, 0)  # Load as grayscale image

        # Apply CLAHE
        image_clahe = clahe.apply(image)

        # Save the CLAHE processed image
        output_path = os.path.join(test_output_directory, filename)
        cv2.imwrite(output_path, image_clahe)


In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:

import os
import cv2
import numpy as np

def circle_crop(img_path, sigmaX=10):
    img = cv2.imread(img_path)
    img = crop_image_from_gray(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    height, width, depth = img.shape

    x = int(width/2)
    y = int(height/2)
    r = np.amin((x, y))

    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x, y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img = cv2.addWeighted(img, 4, cv2.GaussianBlur(img, (0, 0), sigmaX), -4, 128)
    return img

def crop_image_from_gray(img, tol=5):
    if img.ndim == 2:
        mask = img > tol
        return img[np.ix_(mask.any(1), mask.any(0))]
    elif img.ndim == 3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img > tol

        check_shape = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))].shape[0]
        if check_shape == 0:
            return img
        else:
            img1 = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))]
            img2 = img[:, :, 1][np.ix_(mask.any(1), mask.any(0))]
            img3 = img[:, :, 2][np.ix_(mask.any(1), mask.any(0))]
            img = np.stack([img1, img2, img3], axis=-1)
        return img

# Input paths
test_images_folder = '/content/drive/MyDrive/B. Disease Grading/1. Original Images/b. Testing Set'
train_images_folder = '/content/drive/MyDrive/B. Disease Grading/1. Original Images/a. Training Set'
output_test_folder = '/content/drive/MyDrive/B. Disease Grading/1. Original Images/test45'
output_train_folder = '/content/drive/MyDrive/B. Disease Grading/1. Original Images/train45'


# Create the output directories if they don't exist
os.makedirs(output_test_folder, exist_ok=True)
os.makedirs(output_train_folder, exist_ok=True)

# Process test images and save cropped images
for image_name in os.listdir(test_images_folder):
    image_path = os.path.join(test_images_folder, image_name)
    output_image_name = f"{image_name.split('.')[0]}.jpg"
    output_image_path = os.path.join(output_test_folder, output_image_name)

    # Apply circle crop
    cropped_image = circle_crop(image_path)

    # Save the cropped image
    cv2.imwrite(output_image_path, cropped_image)

# Process train images and save cropped images
for image_name in os.listdir(train_images_folder):
    image_path = os.path.join(train_images_folder, image_name)
    output_image_name = f"{image_name.split('.')[0]}.jpg"
    output_image_path = os.path.join(output_train_folder, output_image_name)

    # Apply circle crop
    cropped_image = circle_crop(image_path)

    # Save the cropped image
    cv2.imwrite(output_image_path, cropped_image)


In [ ]:

#prev error - some image files in train_images33 cannot be read: 'NoneType' object has no attribute 'ndim'
#so modified by just not considering that img at all and moving on to the next one
import os
import cv2
import numpy as np

def circle_crop(img_path, sigmaX=10):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Failed to read image: {img_path}")
        return None

    img = crop_image_from_gray(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    height, width, depth = img.shape

    x = int(width/2)
    y = int(height/2)
    r = np.amin((x, y))

    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x, y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    img = cv2.addWeighted(img, 4, cv2.GaussianBlur(img, (0, 0), sigmaX), -4, 128)
    return img

def crop_image_from_gray(img, tol=5):
    if img.ndim == 2:
        mask = img > tol
        return img[np.ix_(mask.any(1), mask.any(0))]
    elif img.ndim == 3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img > tol

        check_shape = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))].shape[0]
        if check_shape == 0:
            return img
        else:
            img1 = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))]
            img2 = img[:, :, 1][np.ix_(mask.any(1), mask.any(0))]
            img3 = img[:, :, 2][np.ix_(mask.any(1), mask.any(0))]
            img = np.stack([img1, img2, img3], axis=-1)
        return img

# Input paths
#test_images_folder = '/content/db_ret_dataset/test_images'
train_images_folder = '/content/drive/MyDrive/B. Disease Grading/1. Original Images/a. Training Set'
#output_test_folder = '/content/drive/MyDrive/db_ret_final_dataset/test_images55'
output_train_folder = '/content/drive/MyDrive/B. Disease Grading/1. Original Images/train45'

# Create the output directories if they don't exist
#os.makedirs(output_test_folder, exist_ok=True)
os.makedirs(output_train_folder, exist_ok=True)
'''
# Process test images and save cropped images
for image_name in os.listdir(test_images_folder):
    image_path = os.path.join(test_images_folder, image_name)
    output_image_name = f"{image_name.split('.')[0]}.png"
    output_image_path = os.path.join(output_test_folder, output_image_name)

    print(f"Processing image: {image_path}")

    # Check if the image file exists
    if not os.path.isfile(image_path):
        print("Image file does not exist.")
        continue

    # Apply circle crop
    cropped_image = circle_crop(image_path)

    # Check if the cropped image is None
    if cropped_image is None:
        print("Failed to crop the image.")
        continue

    # Save the cropped image
    cv2.imwrite(output_image_path, cropped_image)
'''
# Process train images and save cropped images
for image_name in os.listdir(train_images_folder):
    image_path = os.path.join(train_images_folder, image_name)
    output_image_name = f"{image_name.split('.')[0]}.png"
    output_image_path = os.path.join(output_train_folder, output_image_name)

    #print(f"Processing image: {image_path}")

    # Check if the image file exists
    if not os.path.isfile(image_path):
        print(f"Image file does not exist.{image_path}")
        continue

    # Apply circle crop
    cropped_image = circle_crop(image_path)

    # Check if the cropped image is None
    if cropped_image is None:
        print("Failed to crop the image.")
        continue

    # Save the cropped image
    cv2.imwrite(output_image_path, cropped_image)



In [ ]:
img_size = (512, 512)
num_classes = 5

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the input image size
input_shape = (224, 224, 3)  # Adjust the size as needed

# Load and preprocess the IDRID dataset
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,        # Normalize pixel values to [0, 1]
    rotation_range=20,        # Augmentation: Randomly rotate images
    width_shift_range=0.2,    # Augmentation: Randomly shift images horizontally
    height_shift_range=0.2,   # Augmentation: Randomly shift images vertically
    horizontal_flip=True,     # Augmentation: Randomly flip images horizontally
    shear_range=0.2,          # Augmentation: Shear transformations
    zoom_range=0.2,           # Augmentation: Randomly zoom in on images
    fill_mode='nearest'       # Augmentation: Fill mode for image transformations
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/B. Disease Grading/1. Original Images/a. Training Set',    # Path to the training dataset
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical'  # Use 'categorical' for multi-class classification
)

# Define the CNN model
model = models.Sequential([
    # Convolutional layers
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Flatten the output for fully connected layers
    layers.Flatten(),

    # Fully connected layers
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Dropout layer to reduce overfitting
    layers.Dense(num_classes, activation='softmax')  # 'num_classes' is the number of classes in your dataset
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Summary of the model architecture
model.summary()

# Train the model
history = model.fit(
    train_generator,
    epochs=50,  # Adjust the number of epochs as needed
    steps_per_epoch=len(train_generator),
    verbose=1
)

# Save the trained model for future use
model.save('idrid_diabetic_retinopathy_model.h5')

In [ ]:
train_images_folder = '/content/drive/MyDrive/B. Disease Grading/1. Original Images/a. Training Set'
test_images_folder = '/content/drive/MyDrive/B. Disease Grading/1. Original Images/a. Training Set'
'''
train_df = 'train33.csv'
test_df= 'test.csv'
'''
train_df = pd.read_csv('/content/drive/MyDrive/B. Disease Grading/2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv')
test_df= pd.read_csv('/content/drive/MyDrive/B. Disease Grading/2. Groundtruths/b. IDRiD_Disease Grading_Testing Labels.csv')

In [ ]:
train_df["Image name"] = train_df["Image name"].apply(lambda x: x + ".jpg")
test_df["Image name"] = train_df["Image name"].apply(lambda x: x + ".jpg")
train_df['Retinopathy grade'] = train_df['Retinopathy grade'].astype('str')
train_df.head()

In [ ]:
# Define num_classes
num_classes = 10

# Now you can use num_classes in your code
if num_classes > 5:
    print("There are more than 5 classes.")
else:
    print("There are 5 or fewer classes.")


In [ ]:
img_size = (512, 512)
num_classes = 5

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split= 0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
# Load and preprocess the train images
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_images_folder,
    x_col='Image name',
    y_col='Retinopathy grade',
    target_size=img_size,
    batch_size=16,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

valid_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_images_folder,
    x_col='Image name',
    y_col='Retinopathy grade',
    target_size=img_size,
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

# Load and preprocess the test images
# Load and preprocess the test images
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_images_folder,
    x_col='Image name',
    y_col=None,  # No class labels in the test set
    target_size=img_size,
    batch_size=16,
    class_mode=None,
    shuffle=False
)


In [ ]:
from keras.callbacks import LearningRateScheduler

def lr_schedule(epoch):
    if epoch < 30:
        return 0.001
    elif epoch < 45:
        return 0.0001
    else:
        return 0.00001

lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
# Load ResNet50 model
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(img_size[0], img_size[1], 3))

#model architecture
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(train_generator, epochs=79, validation_data=valid_generator)

94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/79
21/21 [==============================] - 94s 2s/step - loss: 1.6331 - accuracy: 0.4230 - val_loss: 3037.1833 - val_accuracy: 0.0000e+00
Epoch 2/79
21/21 [==============================] - 40s 2s/step - loss: 1.3568 - accuracy: 0.4592 - val_loss: 142608.7500 - val_accuracy: 0.0000e+00
Epoch 3/79
21/21 [==============================] - 41s 2s/step - loss: 1.2676 - accuracy: 0.4864 - val_loss: 12496.0078 - val_accuracy: 0.0000e+00
Epoch 4/79
21/21 [==============================] - 41s 2s/step - loss: 1.1866 - accuracy: 0.5347 - val_loss: 2.0356 - val_accuracy: 0.0244
Epoch 5/79
21/21 [==============================] - 41s 2s/step - loss: 1.1004 - accuracy: 0.5559 - val_loss: 3.3989 - val_accuracy: 0.0000e+00
Epoch 6/79
21/21 [==============================] - 41s 2s/step - loss: 1.0137 - accuracy: 0.5831 - val_loss: 3.6477 - val_accuracy: 0.0000e+00
Epoch 7/79
21/21 [==============================] - 41s 2s/step